In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/data-science-london-scikit-learn/trainLabels.csv
/kaggle/input/data-science-london-scikit-learn/train.csv
/kaggle/input/data-science-london-scikit-learn/test.csv


In [2]:
train_data = pd.read_csv("/kaggle/input/data-science-london-scikit-learn/train.csv") # reading 40 feature columns
train_data.head()

,0.29940251144353242,-1.2266241875260637,1.4984250500215328,-1.1761503610375272,5.2898525545597037,0.20829711393323402,2.4044983672405826,1.5945062220589785,-0.051608163273514231,0.66323431039687908,...,-0.85046544625016463,-0.62298999638261954,-1.8330573433160038,0.29302438506869571,3.5526813410266507,0.71761099417552265,3.3059719748508889,-2.7155588147154619,-2.6824085866346223,0.10105047232890663
0,-1.174176,0.332157,0.949919,-1.285328,2.199061,-0.151268,-0.427039,2.619246,-0.765884,-0.093780,...,-0.819750,0.012037,2.038836,0.468579,-0.517657,0.422326,0.803699,1.213219,1.382932,-1.817761
1,1.192222,-0.414371,0.067054,-2.233568,3.658881,0.089007,0.203439,-4.219054,-1.184919,-1.240310,...,-0.604501,0.750054,-3.360521,0.856988,-2.751451,-1.582735,1.672246,0.656438,-0.932473,2.987436
2,1.573270,-0.580318,-0.866332,-0.603812,3.125716,0.870321,-0.161992,4.499666,1.038741,-1.092716,...,1.022959,1.275598,-3.480110,-1.065252,2.153133,1.563539,2.767117,0.215748,0.619645,1.883397
3,-0.613071,-0.644204,1.112558,-0.032397,3.490142,-0.011935,1.443521,-4.290282,-1.761308,0.807652,...,0.513906,-1.803473,0.518579,-0.205029,-4.744566,-1.520015,1.830651,0.870772,-1.894609,0.408332
4,-0.773247,-0.123227,0.047423,-0.210266,10.377793,0.526604,-2.751616,0.315541,0.608603,-0.043421,...,-1.487714,0.792790,-0.540711,0.114115,-0.277477,-0.896411,-2.805207,0.469162,3.614157,0.081689


In [3]:
train_labels = pd.read_csv("/kaggle/input/data-science-london-scikit-learn/trainLabels.csv") #reading binary classification column
train_labels.head()

,1
0,0
1,0
2,1
3,0
4,1


In [4]:
#Let's check the shape of the data before we continue
print(f"Shape of x: {train_data.shape}")
print(f"Shape of y: {train_labels.shape}")

Shape of x: (999, 40)
Shape of y: (999, 1)


In [5]:
test_data = pd.read_csv("/kaggle/input/data-science-london-scikit-learn/test.csv")
test_data.head()

,2.8089094884322816,-0.2428941541280098,-0.54642134078742799,0.25516185655651813,1.7497359401586778,-0.030457614747139938,-1.3220705648016891,3.5780706827705329,-0.66757846469285065,-0.88425685382194696,...,-0.26168811207868709,-0.22437538915802702,-1.6756055243798111,-0.47958360985977283,-0.24438814509352236,-0.67235457360991246,0.51786002310577695,0.010664676497809677,-0.41921432660528302,2.8183870993565581
0,-0.374101,0.537669,0.081063,0.756773,0.915231,2.557282,3.703187,1.673835,-0.764122,-1.228040,...,-0.969463,0.574154,-2.200519,-1.612240,0.179031,-2.924596,0.643610,-1.470939,-0.067408,-0.976265
1,-0.088370,0.154743,0.380716,-1.176126,1.699867,-0.258627,-1.384999,1.093584,1.596633,0.230631,...,-0.769885,-0.005143,1.467490,0.483803,-3.542981,0.814561,-1.652948,1.265866,-1.749248,1.773784
2,-0.685635,0.501283,1.873375,0.215224,-3.983468,-0.103637,4.136113,-0.225431,-1.515015,-1.071763,...,0.968609,2.386412,-0.131219,0.285646,2.302069,1.255588,-1.563090,-0.125258,-1.030761,-2.945329
3,0.350867,0.721897,-0.477104,-1.748776,-2.627405,1.075433,4.954253,-3.293501,-0.760369,0.204360,...,0.260553,-2.045650,-2.173227,0.372992,0.450700,-0.211657,1.301359,-0.522164,2.484883,0.039213
4,-0.424627,1.536196,-1.037752,-0.156466,-2.945038,-0.471607,3.494966,-2.763629,0.819540,0.209529,...,-0.727066,-0.809620,4.504230,-0.481077,-2.923646,-0.468034,-0.846214,1.197350,-5.615563,2.049134


In [6]:
#Standerize the features
Scaler = StandardScaler()
train_data = Scaler.fit_transform(train_data)
test_data = Scaler.fit_transform(test_data)

In [7]:
y = train_labels.values.ravel()
model = RandomForestClassifier()
model.fit(train_data, y)

RandomForestClassifier()

In [8]:
y_pred = model.predict(test_data)
print(y_pred)

[0 0 0 ... 1 0 1]


In [9]:
#Since we don't have GT for test_data we can't use accuracy_score. 
#Instead, we can use cross validation method
# Perform 5-fold cross-validation
cv_scores = cross_val_score(model, train_data, y, cv=5, scoring='accuracy')
print(f"Cross-validation accuracy score: {cv_scores}")

Cross-validation accuracy score: [0.875      0.88       0.885      0.845      0.84924623]
